In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_predict,cross_val_score
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import svm 
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import collections
import os
import pandas as pd
import networkx as nx
import math
import re

In [2]:
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score


In [7]:
# edge embbeding functions
def avg_sum(v1, v2):
    return (np.array(v1)+np.array(v2))/2
def mult(v1, v2):
    return np.array(v1)*np.array(v2)
def w_l1(v1, v2):
    return np.abs(np.array(v1)-np.array(v2))
def w_l2(v1, v2):
    return (np.array(v1)-np.array(v2))**2
def nw_l1(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(str(n1))]
    neig2 = [n for n in graph.neighbors(str(n2))]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return np.abs((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))
def nw_l2(v1, v2, graph, n1, n2, embs ):
    neig1 = [n for n in graph.neighbors(n1)]
    neig2 = [n for n in graph.neighbors(n2)]
    sum1 = np.zeros(len(v1))
    sum2 = np.zeros(len(v2))
    for n in neig1:
        sum1 += np.array(embs[int(n)])
    for n in neig2:
        sum2 += np.array(embs[int(n)])
    return ((sum1+np.array(v1))/(len(neig1)+1)+(sum2+np.array(v2))/(len(neig2)+1))**2

In [5]:
# data load
Graph_train = nx.read_weighted_edgelist("./edge_list_binary_train_2018.txt", delimiter=' ',nodetype=int)
Graph_test_2018 = nx.read_weighted_edgelist("./edge_list_binary.txt", delimiter=' ',nodetype=int)

In [3]:
Graph_test_2018 = nx.read_weighted_edgelist("./edge_list_weighted.txt", delimiter=' ',nodetype=int)
# Graph_test_2018

In [7]:
Graph_dir = Graph_test_2018.to_directed()

In [11]:
edge_list_dir = list(Graph_dir.edges.data()) 

In [22]:
# edge_list_dir
edge_list_dir_save = [[e[0],e[1],int(list(e[2].values())[0])] for e in edge_list_dir]

In [27]:
new = nx.convert_node_labels_to_integers(Graph_test_2018)

In [77]:
new.number_of_nodes()

46312

In [38]:
old = list(Graph_dir.edges())

In [43]:
zipped = dict(zip(old,int_edges))

In [63]:
first = [(key[0],val[0]) for key,val in zipped.items()]
second = [(key[1],val[1]) for key,val in zipped.items()]

In [59]:
import json

In [70]:
df = pd.DataFrame(first)
df_2 = pd.DataFrame(second)

In [71]:
df = df.drop_duplicates()
df_2 = df_2.drop_duplicates()

In [73]:
merged = pd.merge(df, df_2, on=[0,1], how='inner')

In [75]:
len(merged)

123

In [82]:
df_first = df[~df.isin(merged)].dropna()

In [83]:
len(df)

39431

In [84]:
len(df_first)

39431

In [37]:
int_edges = list(new.edges())

In [25]:
with open('dir_edge_list_weighted.txt', 'w') as file:
    for ind in range(len(edge_list_dir_save)):
        file.writelines(str(edge_list_dir_save[ind][0]) + ' ' + str(edge_list_dir_save[ind][1]) + ' '+ str(edge_list_dir_save[ind][2]) +'\n')

In [45]:
f_train = open('./edges_train_binary.txt')
f_2019 = open('./edges_2019_binary.txt')

edges_train = []
edges_2019 = []

for line in f_train:
    edges_train.append(line.split(' '))
for line in f_2019:
    edges_2019.append(line.split(' '))
f_train.close()
f_2019.close()

for i in range(len(edges_train)):
    edges_train[i][2] = edges_train[i][2][:-1]

for i in range(len(edges_2019)):
    edges_2019[i][2] = edges_2019[i][2][:-1]

In [15]:
edges_2019 = [edge for edge in edges_train if edge not in edges_2018]

In [28]:
dataframe = pd.DataFrame(edges_train)
len(dataframe)

117228

In [29]:
dataframe_2 = pd.DataFrame(edges_2018)
len(dataframe_2)

180473

In [43]:
dataframe.to_csv('edges_train_binary.txt', sep=' ', index=False, header=False)

In [30]:
data_merge = pd.merge(dataframe, dataframe_2, on=[0,1], how='inner')
len(data_merge)

117228

In [33]:
data_merge = data_merge.drop(columns=['2_y'])

In [35]:
data_merge = data_merge.rename(columns={0: 0, 1: 1, '2_x': 2})

In [36]:
data_2019 = dataframe_2[~dataframe_2.isin(data_merge)].dropna()

In [40]:
data_2019.to_csv('edges_2019_binary.txt',sep=' ', index=False, header=False)

In [46]:
print(len(edges_2019))
print(len(edges_train))

63245
117228


In [86]:
# edges_train

In [8]:
#загрузка ребер с негатив семплинг
f_train = open('./edge_list_train_NegSamp.txt')
f_2019 = open('./edge_list_2019_NegSamp.txt')

edges_train = []
edges_2019 = []

for line in f_train:
    edges_train.append(line.split(' '))
for line in f_2019:
    edges_2019.append(line.split(' '))
f_train.close()
f_2019.close()

for i in range(len(edges_train)):
    edges_train[i][2] = edges_train[i][2][:-1]

for i in range(len(edges_2019)):
    edges_2019[i][2] = edges_2019[i][2][:-1]

In [9]:
edges_train_dict = {(e[0], e[1]) : e[2] for e in edges_train}
edges_2019_dict = {(e[0], e[1]) : e[2] for e in edges_2019}

In [3]:
%%time
# Calculating embeddings (finding best parameters: 'p' and 'q')
# Source: https://github.com/aditya-grover/node2vec
# values = [0.25,0.5,1,2,4] # d=64
# for x in values:
#     for y in values:
%run /home/natalia-s/Documents/makarov_jcdl/jcdl2018/Core_Code_for_XLSX_results/node2vec-master/src/main.py --input ./edge_list_weighted.txt --output ./Data/emb_done/out.txt --walk-length 60 --num-walks 10 --q 2 --p 0.5 --weighted

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
CPU times: user 9min 57s, sys: 1.21 s, total: 9min 58s
Wall time: 3min 32s


In [12]:
%%time
# Calculating embeddings (finding best parameters: 'p' and 'q')
# Source: https://github.com/aditya-grover/node2vec
# values = [0.25,0.5,1,2,4] # d=64
# for x in values:
#     for y in values:
%run /home/natalia-s/Documents/makarov_jcdl/jcdl2018/Core_Code_for_XLSX_results/node2vec-master/src/main.py --input ./edge_list_weighted_train_2018.txt --output ./Data/emb_done/out_2018.txt --walk-length 60 --num-walks 10 --q 2 --p 0.5 --weighted

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
CPU times: user 6min 29s, sys: 849 ms, total: 6min 30s
Wall time: 2min 25s


In [10]:
nodes_train = []
for e in edges_train:
    nodes_train.extend(e[:2]) # экстендим парами и убираем дубликаты вершин
nodes_train = set(nodes_train)

nodes_2019 = []
for e in edges_2019:
    nodes_2019.extend(e[:2])
nodes_2019 = set(nodes_2019)


In [10]:
# negative sampling
k = 0
neg_sample_train = []
while k != len(edges_train):
    i = np.random.choice(list(nodes_train))
    j = np.random.choice(list(nodes_train))
    if i!=j and (i,j) not in edges_train_dict.keys():
        neg_sample_train.append((i,j))
        k +=1

In [16]:
with open('neg_sample_train.txt', 'w') as file:
    for ind in range(len(neg_sample_train)):
        file.writelines(neg_sample_train[ind][0] + ' ' + neg_sample_train[ind][1] + '\n')

In [11]:
for e_e in neg_sample_train:
    edges_train_dict[e_e] = 0
    edges_train.append([e_e[0],e_e[1], '0'])

In [22]:
with open('edge_list_train_NegSamp.txt', 'w') as file:
    for ind in range(len(edges_train)):
        file.writelines(edges_train[ind][0] + ' ' + edges_train[ind][1] + ' ' + edges_train[ind][2] + '\n')

In [50]:
k = 0
neg_sample_2019 = []
while k != len(edges_2019):
    i = np.random.choice(list(nodes_2019))
    j = np.random.choice(list(nodes_2019))
    if i!=j and (i,j) not in edges_2019_dict.keys():
        neg_sample_2019.append((i,j))
        k +=1

In [51]:
with open('neg_sample_2019.txt', 'w') as file:
    for ind in range(len(neg_sample_2019)):
        file.writelines(neg_sample_2019[ind][0] + ' ' + neg_sample_2019[ind][1] + '\n')

In [52]:
for e_e in neg_sample_2019:
    edges_2019_dict[e_e] = 0
    edges_2019.append([e_e[0],e_e[1], '0'])

In [53]:
with open('edge_list_2019_NegSamp.txt', 'w') as file:
    for ind in range(len(edges_2019)):
        file.writelines(edges_2019[ind][0] + ' ' + edges_2019[ind][1] + ' ' + edges_2019[ind][2] + '\n')

In [11]:
X_train = []
y_train = []
X_test = []
y_test = []
# f_emb_train = open('./Data/emb_done/out_2018.txt')
f_emb = open('./Data/emb_done/out.txt')
emb = []
# emb_2018 = []
for line in f_emb:
    emb.append(line.split(' '))
# for line in f_emb_2018:
#     emb_2018.append(line.split(' '))
f_emb.close()
# f_emb_2018.close()


for i in range(len(emb)):
    emb[i][len(emb[i])-1] = emb[i][len(emb[i])-1][:-1] # для удаления /n на конце последнего числа эмбеддинга
# for i in range(len(emb_2018)):
#     emb_2018[i][len(emb_2018[i])-1] = emb_2018[i][len(emb_2018[i])-1][:-1]
    
emb_dict = {} 
for i in range(1,len(emb)):
    emb_dict[int(emb[i][0])] = [float(j) for j in emb[i][1:]] # собираем эмбеддинг в ключ-значение
# emb_2018_dict = {} 
# for i in range(1,len(emb_2018)):
#     emb_2018_dict[int(emb_2018[i][0])] = [float(j) for j in emb_2018[i][1:]]

for e in edges_2019:
    if (e[0] in nodes_2019) and (e[1] in nodes_2019):
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w_19 = int(e[2])
        y_test.append(w_19)
#         res = nw_l2(emb1, emb2, Graph_test_2018,int(e[0]),int(e[1]),emb_2018_dict)
        res = mult(emb1, emb2)
#         try:
#             w_train = float(edges_2018_dict[(e[0], e[1])])
#         except  KeyError :
#             w_train = 0
        #res = np.append(res,w_16)
        X_test.append(res)

for e in edges_train:
    if (e[0] in nodes_train) and (e[1] in nodes_train):
        emb1 = emb_dict[int(e[0])]
        emb2 = emb_dict[int(e[1])]
        w_train = int(e[2])
        y_train.append(w_train)
#         res = nw_l2(emb1, emb2, Graph_train,int(e[0]),int(e[1]),emb_train_dict)
        res = mult(emb1, emb2)
#         try:
#             w_15 = float(edges_2015_dict[(e[0], e[1])])
#         except  KeyError :
#             w_15 = 0
        #res = np.append(res,w_15)
        X_train.append(res)

In [85]:
# emb_dict

In [65]:
len(X_test)

126490

In [75]:
def model_svm(X_train, y_train, X_test, y_test):

    # SVC
#     y_train = label_binarize(y_train, classes=[0, 1, 2, 3, 4])
    svm_model = svm.SVC()
#     svm_model = svm.SVC(probability=True)
    svm_model.fit(X_train, y_train)
#     print(X_train, "X_train")
#     print(y_train, 'y_train')
                     
    y_pred_train = svm_model.predict(X_train)
    y_pred_test = svm_model.predict(X_test)
#     y_pred_train_proba = svm_model.predict_proba(X_train)
#     y_pred_test_proba = svm_model.predict_proba(X_test)
#     print(y_pred_train_proba[0])

#     print(y_pred_train, 'y_pred_train')
#     labels = [0,1]
    
    precision_train = precision_score(y_train, y_pred_train, average=None)
    precision_test = precision_score(y_test, y_pred_test, average=None)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [58]:
def model_rf(X_train, y_train, X_test, y_test):
    
    # RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    
    
    y_pred_train = rf.predict(X_train)
    y_pred_test = rf.predict(X_test)
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [2]:
def model_gbc(X_train, y_train, X_test, y_test):
    
    # GradientBoosting
    params = {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2,
              'learning_rate': 0.01}
    clf = GradientBoostingClassifier(**params)
    clf.fit(X_train, y_train)
                     
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train)
    logloss_test = log_loss(y_test, y_pred_test)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  

In [73]:
def model_xgb(X_train, y_train, X_test, y_test):
#     param = {
#    'max_depth': 3,
#    'eta': 0.3, 
#    'silent': 1, 
#    'objective': 'multi:softprob',
#    'num_class': 2}
#     num_round = 20
    param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
    num_round = 10
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)
#     clf = xgboost.XGBClassifier()
    clf = xgb.train(param, dtrain, num_round)
#     clf = XGBRegressor()
#     clf.fit(X_train, y_train, verbose=False)
    evallist = [(dtest, 'eval'), (dtrain, 'train')]
    clf = xgb.train(param, dtrain, num_round)
    
#     clf.fit(X_train, y_train)
#     predictions = [round(value) for value in y_pred]
    y_pred_train_prob = clf.predict(dtrain)
    y_pred_train = [int(round(value)) for value in y_pred_train_prob]
    y_pred_test_prob = clf.predict(dtest)
    y_pred_test = [int(round(value)) for value in y_pred_test_prob]
#     print(y_pred_test)
    
#     y_pred_train = clf.predict(X_train)
#     y_pred_train = np.asarray([np.argmax(line) for line in y_pred_train])
#     print(y_pred_train[0])
#     y_pred_test = clf.predict(X_test)
#     y_pred_test = np.asarray([np.argmax(line) for line in y_pred_test])

    
    precision_train = precision_score(y_train, y_pred_train)
    precision_test = precision_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    f1_macro_train = f1_score(y_train, y_pred_train, average='macro')
    f1_macro_test = f1_score(y_test, y_pred_test, average='macro')
    f1_micro_train = f1_score(y_train, y_pred_train, average='micro')
    f1_micro_test = f1_score(y_test, y_pred_test, average='micro')
    logloss_train = log_loss(y_train, y_pred_train_prob)
    logloss_test = log_loss(y_test, y_pred_test_prob)
    roc_auc_train = roc_auc_score(y_train, y_pred_train)
    roc_auc_test = roc_auc_score(y_test, y_pred_test)

    return precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  
    

In [ ]:
%%time
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_svm(X_train, y_train, X_test, y_test)
print('Precision train: ' + str(precision_train))
print('Accuracy train: ' + str(accuracy_train))
print('F-1 (macro) train: ' + str(f1_macro_train))
print('F-1 (micro) train: ' + str(f1_micro_train))
print('Logloss train: ' + str(logloss_train))
print('ROC-AUC train: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision test: ' + str(precision_test))
print('Accuracy test: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))

In [28]:
%%time
#model_rf
#nw_l2
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_rf(X_train, y_train, X_test, y_test)
print('Precision train: ' + str(precision_train))
print('Accuracy train: ' + str(accuracy_train))
print('F-1 (macro) train: ' + str(f1_macro_train))
print('F-1 (micro) train: ' + str(f1_micro_train))
print('Logloss train: ' + str(logloss_train))
print('ROC-AUC train: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision test: ' + str(precision_test))
print('Accuracy test: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))



Precision train: 0.9997778499290828
Accuracy train: 0.9989678233869042
F-1 (macro) train: 0.998967822709046
F-1 (micro) train: 0.9989678233869042
Logloss train: 0.03565020591113815
ROC-AUC train: 0.9989678233869042
-------------------------------------------------
Precision train: 0.8361221779548472
Accuracy train: 0.5771334216198545
F-1 (macro) train: 0.5034299363620667
F-1 (micro) train: 0.5771334216198545
Logloss train: 14.605309230676509
ROC-AUC train: 0.5771334216198545
CPU times: user 47.4 s, sys: 96.5 ms, total: 47.5 s
Wall time: 47.6 s


In [71]:
%%time
#2019 mult
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_rf(X_train, y_train, X_test, y_test)
print('Precision train: ' + str(precision_train))
print('Accuracy train: ' + str(accuracy_train))
print('F-1 (macro) train: ' + str(f1_macro_train))
print('F-1 (micro) train: ' + str(f1_micro_train))
print('Logloss train: ' + str(logloss_train))
print('ROC-AUC train: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision test: ' + str(precision_test))
print('Accuracy test: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))


Precision train: 0.9996844188188735
Accuracy train: 0.9997526188282663
F-1 (macro) train: 0.9997526188271142
F-1 (micro) train: 0.9997526188282663
Logloss train: 0.008544369160993838
ROC-AUC train: 0.9997526188282663
-------------------------------------------------
Precision test: 0.9968255973720368
Accuracy test: 0.9974069096371254
F-1 (macro) train: 0.9974069087496245
F-1 (micro) train: 0.9974069096371254
Logloss train: 0.08956343882215431
ROC-AUC train: 0.9974069096371255
CPU times: user 1min 5s, sys: 79.8 ms, total: 1min 5s
Wall time: 1min 5s


In [13]:
%%time
#model_rf
#mult embed
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_rf(X_train, y_train, X_test, y_test)
print('Precision train: ' + str(precision_train))
print('Accuracy train: ' + str(accuracy_train))
print('F-1 (macro) train: ' + str(f1_macro_train))
print('F-1 (micro) train: ' + str(f1_micro_train))
print('Logloss train: ' + str(logloss_train))
print('ROC-AUC train: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision test: ' + str(precision_test))
print('Accuracy test: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))


Precision train: 0.9997696835281071
Accuracy train: 0.9997782099839628
F-1 (macro) train: 0.9997782099839467
F-1 (micro) train: 0.9997782099839628
Logloss train: 0.007660447852331377
ROC-AUC train: 0.9997782099839628
-------------------------------------------------
Precision test: 0.9851842842848376
Accuracy test: 0.9863026602317244
F-1 (macro) train: 0.9863026420372716
F-1 (micro) train: 0.9863026602317244
Logloss train: 0.4730952924192307
ROC-AUC train: 0.9863026602317244
CPU times: user 1min 6s, sys: 128 ms, total: 1min 6s
Wall time: 1min 6s


In [68]:
%%time
#model_xgb
#nw_l2
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_xgb(X_train, y_train, X_test, y_test)
print('Precision: ' + str(precision_train))
print('Accuracy: ' + str(accuracy_train))
print('F-1 (macro): ' + str(f1_macro_train))
print('F-1 (micro): ' + str(f1_micro_train))
print('Logloss: ' + str(logloss_train))
print('ROC-AUC: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision train: ' + str(precision_test))
print('Accuracy train: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))

Precision: 0.8814461273156414
Accuracy: 0.8736010168219197
F-1 (macro): 0.8735876489822981
F-1 (micro): 0.8736010168219197
Logloss: 0.35416527515648294
ROC-AUC: 0.8736010168219195
-------------------------------------------------
Precision train: 0.5696553377602767
Accuracy train: 0.6062485801200179
F-1 (macro) train: 0.5770673062661725
F-1 (micro) train: 0.6062485801200179
Logloss train: nan
ROC-AUC train: 0.606248580120018
CPU times: user 32.9 s, sys: 1.98 s, total: 34.9 s
Wall time: 34.9 s


In [74]:
%%time
#model_xgb
#2019 mult
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_xgb(X_train, y_train, X_test, y_test)
print('Precision: ' + str(precision_train))
print('Accuracy: ' + str(accuracy_train))
print('F-1 (macro): ' + str(f1_macro_train))
print('F-1 (micro): ' + str(f1_micro_train))
print('Logloss: ' + str(logloss_train))
print('ROC-AUC: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision train: ' + str(precision_test))
print('Accuracy train: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))

Precision: 0.9874320968821769
Accuracy: 0.9906379022076637
F-1 (macro): 0.9906378009649159
F-1 (micro): 0.9906379022076637
Logloss: 0.029438242443460647
ROC-AUC: 0.9906379022076637
-------------------------------------------------
Precision train: 0.9883468919810283
Accuracy train: 0.9916594197169737
F-1 (macro) train: 0.9916593237761457
F-1 (micro) train: 0.9916594197169737
Logloss train: 0.02766281220040771
ROC-AUC train: 0.9916594197169736
CPU times: user 51.8 s, sys: 1.36 s, total: 53.2 s
Wall time: 15.1 s


In [14]:
%%time
#mult
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_xgb(X_train, y_train, X_test, y_test)
print('Precision: ' + str(precision_train))
print('Accuracy: ' + str(accuracy_train))
print('F-1 (macro): ' + str(f1_macro_train))
print('F-1 (micro): ' + str(f1_micro_train))
print('Logloss: ' + str(logloss_train))
print('ROC-AUC: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision train: ' + str(precision_test))
print('Accuracy train: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))

Precision: 0.9869626766353873
Accuracy: 0.9900407752414099
F-1 (macro): 0.9900406757596212
F-1 (micro): 0.9900407752414099
Logloss: 0.03003821213053024
ROC-AUC: 0.99004077524141
-------------------------------------------------
Precision train: 0.9256401371556241
Accuracy train: 0.9560266632681897
F-1 (macro) train: 0.9559705636259489
F-1 (micro) train: 0.9560266632681897
Logloss train: 0.1347615421745948
ROC-AUC train: 0.9560266632681899
CPU times: user 58.6 s, sys: 2.17 s, total: 1min
Wall time: 21.6 s


In [12]:
%time
#model_gbc
precision_train, precision_test, accuracy_train, accuracy_test, f1_macro_train, f1_macro_test, f1_micro_train, f1_micro_test, logloss_train, logloss_test, roc_auc_train, roc_auc_test, y_pred_train, y_pred_test  = model_gbc(X_train, y_train, X_test, y_test)
print('Precision: ' + str(precision_train))
print('Accuracy: ' + str(accuracy_train))
print('F-1 (macro): ' + str(f1_macro_train))
print('F-1 (micro): ' + str(f1_micro_train))
print('Logloss: ' + str(logloss_train))
print('ROC-AUC: ' + str(roc_auc_train))
print('-------------------------------------------------')
print('Precision train: ' + str(precision_test))
print('Accuracy train: ' + str(accuracy_test))
print('F-1 (macro) train: ' + str(f1_macro_test))
print('F-1 (micro) train: ' + str(f1_micro_test))
print('Logloss train: ' + str(logloss_test))
print('ROC-AUC train: ' + str(roc_auc_test))

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.81 µs
Precision: 0.9867962744815546
Accuracy: 0.9899895929300168
F-1 (macro): 0.9899894852372865
F-1 (micro): 0.9899895929300168
Logloss: 0.34575252487311825
ROC-AUC: 0.9899895929300168
-------------------------------------------------
Precision train: 0.9910365469439193
Accuracy train: 0.9928610957387936
F-1 (macro) train: 0.9928610710979033
F-1 (micro) train: 0.9928610957387936
Logloss train: 0.24657261487502763
ROC-AUC train: 0.9928610957387936


In [35]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

234456
234456
360946
360946


In [64]:
def model_lasso(X_train, y_train, X_test, y_test):
    MSE_train= []
    CV_train = []
    MSE_test= []
    CV_test = []
    MAE_train = []
    MAPE_train =[]
    MAE_test = []
    MAPE_test =[]
    
    #  Lasso
    lasso = Lasso(alpha = 0.016)
    y_pred_lasso_train = lasso.fit(X_train, y_train).predict(X_train)
    y_pred_lasso = lasso.fit(X_train, y_train).predict(X_test)
    
    MSE_train.append(np.mean((y_pred_lasso_train - y_train) ** 2))
    MSE_test.append(np.mean((y_pred_lasso - y_test) ** 2))
     
    scores1 = cross_val_score(lasso, X_train, y_train, cv=10,scoring='neg_mean_squared_error')
    CV_train.append([scores1.mean(), scores1.std() * 2])
    scores2 = cross_val_score(lasso, X_test, y_test, cv=10,scoring='neg_mean_squared_error')
    CV_test.append([scores2.mean(), scores2.std() * 2])
    
    MAE_train.append(metrics.mean_absolute_error(y_train, y_pred_lasso_train))
    MAE_test.append(metrics.mean_absolute_error(y_test, y_pred_lasso))
    
    MAPE_train.append(np.mean(np.abs((y_train - y_pred_lasso_train) / y_train)) * 100)
    MAPE_test.append(np.mean(np.abs((y_test - y_pred_lasso) / y_test)) * 100)

    return MSE_train[0], CV_train[0], MSE_test[0], CV_test[0], MAE_train[0], MAPE_train[0], MAE_test[0], MAPE_test[0]

In [43]:
MSE_train= []
CV_train = []
MSE_test= []
CV_test = []
MAE_train = []
MAPE_train =[]
MAE_test = []
MAPE_test =[]

# X_train = []
# y_train = []
# X_test = []
# y_test = []
# f_emb_2015 = open('/Users/olgagerasimova/Desktop/Data/emb_new_nw/out15nw_'+str(x)+'_'+str(y)+'.txt')
# f_emb_2016 = open('/Users/olgagerasimova/Desktop/Data/emb_new_nw/out16nw_'+str(x)+'_'+str(y)+'.txt')
# emb_2015 = []
# emb_2016 = []
# for line in f_emb_2015:
#     emb_2015.append(line.split(' '))
# for line in f_emb_2016:
#     emb_2016.append(line.split(' '))
# f_emb_2015.close()
# f_emb_2016.close()
# for i in range(len(emb_2016)):
#     emb_2016[i][len(emb_2016[i])-1] = emb_2016[i][len(emb_2016[i])-1][:-1]
# for i in range(len(emb_2015)):
#     emb_2015[i][len(emb_2015[i])-1] = emb_2015[i][len(emb_2015[i])-1][:-1]
# emb_2015_dict = {} 
# for i in range(1,len(emb_2015)):
#     emb_2015_dict[int(emb_2015[i][0])] = [float(j) for j in emb_2015[i][1:]]
# emb_2016_dict = {} 
# for i in range(1,len(emb_2016)):
#     emb_2016_dict[int(emb_2016[i][0])] = [float(j) for j in emb_2016[i][1:]]

# for e in edges_2017:
#     if (e[0] in nodes_2016) and (e[1] in nodes_2016):
#         emb1 = emb_2016_dict[int(e[0])]
#         emb2 = emb_2016_dict[int(e[1])]
#         w_17 = float(e[2])
#         y_test.append(w_17)
#         res = avg_sum(emb1, emb2)
#         try:
#             w_16 = float(edges_2016_dict[(e[0], e[1])])
#         except  KeyError :
#             w_16 = 0
#         #res = np.append(res,w_16)
#         X_test.append(res)

# for e in edges_2016:
#     if (e[0] in nodes_2015) and (e[1] in nodes_2015):
#         emb1 = emb_2015_dict[int(e[0])]
#         emb2 = emb_2015_dict[int(e[1])]
#         w_16 = float(e[2])
#         y_train.append(w_16)
#         res = avg_sum(emb1, emb2)
#         try:
#             w_15 = float(edges_2015_dict[(e[0], e[1])])
#         except  KeyError :
#             w_15 = 0
#         #res = np.append(res,w_15)
#         X_train.append(res)

mse_train, cv_train, mse_test,cv_test, mae_train, mape_train, mae_test,mape_test = model_lasso(X_train, y_train, X_test, y_test)
MSE_train.append(mse_train)
CV_train.append( cv_train)
MSE_test.append(mse_test)
CV_test.append(cv_test)
MAE_train.append(mae_train)
MAPE_train.append(mape_train)
MAE_test.append(mae_test)
MAPE_test.append(mape_test)

/home/natalia-s/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in true_divide
/home/natalia-s/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in true_divide


In [44]:
# http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
f_res = open('res_pq_lasso_negsamp.txt','w')
for i in range(len(MSE_train)):
    f_res.write(str(MSE_train[i])+"  "+str(CV_train[i][0]) +"  "+ str(CV_train[i][1])+"  "+str(MSE_test[i])+"  "\
                +str(CV_test[i][0]) +"  "+ str(CV_test[i][1])+"  "+str(MAE_train[i])+"  "+str(MAPE_train[i]) +"  "\
                + str(MAE_test[i])+ "  "+str(MAPE_test[i])+'\n')
f_res.close()

In [46]:
MSE_train

[0.0906746807176951]

In [47]:
MSE_test

[0.11878197790002845]

In [48]:
MAPE_test

[inf]